## 获取大模型

In [69]:
from itertools import chain

from langchain_classic.chains.summarize.refine_prompts import prompt_template
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from nbformat.v4 import output_from_msg

llm = ChatOpenAI(
    model="google/gemma-3n-e4b",
    openai_api_key="lm-studio",
    openai_api_base="http://localhost:1234/v1"
)

print(llm.invoke("10个字回答我：什么是大模型"))

content='**海量数据训练，智能生成，多功能AI。**\n' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 19, 'total_tokens': 35, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_provider': 'openai', 'model_name': 'google/gemma-3n-e4b', 'system_fingerprint': 'google/gemma-3n-e4b', 'id': 'chatcmpl-5gl5racs1f3aimrbso15hb', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--64b46e53-9e52-49c3-a064-2c3622e0cff2-0' usage_metadata={'input_tokens': 19, 'output_tokens': 16, 'total_tokens': 35, 'input_token_details': {}, 'output_token_details': {}}


# 使用提示词模版

chain = prompt | llm

chain: 这是一个 LangChain 中用于定义一系列步骤的链。它将 prompt 和 LLM 连接起来，形成一个完整的流程。
prompt | llm: 这使用了 LangChain 的 | 运算符（即 pipe 操作符）。它将 prompt 和 llm 连接起来，表示：
首先使用 prompt 模板创建 prompt。
然后将生成的 prompt 发送给 llm（大型语言模型，例如 OpenAI 的 GPT 模型）。
LLM 会根据 prompt 生成回复。
chain = ...: 将构建好的链赋值给变量 chain。

In [70]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        ("system","你是世界级的专家"),
        ("user","{input}")
    ])

chain = prompt | llm
message = chain.invoke({"input":"两句话，大模型的langchain是啥"})
print(message.content)

Langchain 是一个强大的开源框架，旨在简化大型语言模型 (LLM) 的应用开发，提供了一系列工具和模块，用于连接 LLM 与其他数据源和工具。 简单来说，它是一个构建基于 LLM 的应用的瑞士军刀。



# 使用输出解析器

In [36]:
from langchain_core.output_parsers import JsonOutputParser

output_parser =JsonOutputParser()

chain = prompt | llm | output_parser

message = chain.invoke({"input":"大模型的langchain是什么？，用JSON格式，5句话内回答，问题用question,回答用answer"})
print(message)


{'question': 'Langchain是什么？', 'answer': 'Langchain 是一个旨在简化大型语言模型 (LLM) 应用开发的框架。它提供了一系列工具，用于连接 LLM 与其他数据源和工具，构建复杂的链式应用。核心功能包括模型集成、提示管理、链（chains）和代理（agents）。它旨在提高 LLM 应用的可扩展性和实用性，降低开发难度。Langchain 促进了 LLM 在各种场景中的应用，例如问答、文本生成和数据分析。'}


In [ ]:
# 使用向量存储

In [68]:
# ===============================
# 1. 加载网页
# ===============================
from langchain_community.document_loaders import WebBaseLoader
import bs4

url = "https://www.gov.cn/zhengce/zhengceku/202504/content_7021191.htm"

loader = WebBaseLoader(
    web_path=url,
    bs_kwargs=dict(parse_only=bs4.SoupStrainer(id="UCAP-CONTENT"))
)
docs = loader.load()

print("网页加载完成：", len(docs), "个文档")


# ===============================
# 2. 文本分块
# ===============================
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50
)

documents = text_splitter.split_documents(docs)
texts = [d.page_content for d in documents]

print("文本分块完成：", len(texts), "块")


# ===============================
# 3. LM Studio Embedding（自定义类）
# ===============================
import requests
import numpy as np
from langchain_core.embeddings import Embeddings

class LMStudioEmbeddings(Embeddings):
    def __init__(self, base_url="http://localhost:1234/v1", model="text-embedding-nomic-embed-text-v1.5"):
        self.base_url = base_url
        self.model = model

    def embed_query(self, text: str):
        payload = {"input": text, "model": self.model}
        r = requests.post(f"{self.base_url}/embeddings", json=payload)
        return r.json()["data"][0]["embedding"]

    def embed_documents(self, texts):
        vectors = []
        for t in texts:
            vectors.append(self.embed_query(t))
        return vectors


embedder = LMStudioEmbeddings(
    base_url="http://localhost:1234/v1",
    model="text-embedding-nomic-embed-text-v1.5"
)

print("Embedding 模型准备完成")


# ===============================
# 4. 生成向量（逐条，不走 LangChain 的批处理）
# ===============================
vectors = embedder.embed_documents(texts)
vectors = np.array(vectors)

print("向量生成完成：", vectors.shape)


# ===============================
# 5. 使用 FAISS 构建向量库（正确格式）
# ===============================
from langchain_community.vectorstores import FAISS

text_embedding_pairs = list(zip(texts, vectors))

vector_store = FAISS.from_embeddings(
    text_embeddings=text_embedding_pairs,   # ← 关键：传二元组列表
    embedding=embedder,
    metadatas=[{} for _ in texts],
)

print("向量库构建完成！")


# ===============================
# 6. 检索
# ===============================
retriever = vector_store.as_retriever(search_kwargs={"k": 4})

query = "这个政策的主要内容是什么？"

# 新版本写法
relevant_docs = retriever.invoke(query)

context = "\n\n".join([d.page_content for d in relevant_docs])

print("检索完成：", len(relevant_docs), "条")


# ===============================
# 7. 使用 LM Studio LLM 回答（ChatOpenAI）
# ===============================
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

llm = ChatOpenAI(
    model="google/gemma-3n-e4b",
    openai_api_key="lm-studio",
    openai_api_base="http://localhost:1234/v1",
    temperature=0
)

prompt = ChatPromptTemplate.from_messages([
    ("system", "你是专业政策分析助手，请基于给定的上下文回答问题，不允许编造内容。"),
    ("user", """
问题：{question}

上下文：
{context}

请总结回答。
""")
])

chain = prompt | llm

response = chain.invoke({
    "question": query,
    "context": context
})

print("\n================== 最终回答 ==================\n")
print(response.content)

网页加载完成： 1 个文档
文本分块完成： 5 块
Embedding 模型准备完成
向量生成完成： (5, 768)
向量库构建完成！
检索完成： 4 条

================== 最终回答 ==================

这个政策的主要内容是：**在2025年5月（第五个“民法典宣传月”）以“民法典进企业”为重点，开展一系列普法宣传活动，旨在提高企业诚信守法经营意识和能力，助力企业防范化解风险，促进经济高质量发展。**

具体来说，政策强调：

*   **以习近平法治思想为指导，深入学习宣传民法典。**
*   **重点开展“民法典进企业”活动，并结合其他领域（如机关、乡村、社区等）开展宣传。**
*   **突出学习宣传习近平法治思想，重点强调民法典的基本原则和相关规定。**
*   **加强与各部门的协调配合，形成广泛参与的普法工作格局。**
*   **鼓励创新宣传方式，运用传统和现代化手段相结合，提升宣传效果。**
*   **强调反对形式主义，注重服务企业，不给企业增加负担。**

总而言之，该政策旨在通过广泛的宣传活动，让民法典深入企业，提升企业法律意识，营造良好的法治环境。



# Rag检索增强生成
| 特性 | FAISS | Chroma | Pinecone |
|------|-------|--------|----------|
| **部署形态** | 本地/自托管（CPU/GPU） | 本地 Docker / Cloud（轻量） | 完全托管 SaaS |
| **开源/商业** | 完全开源（MIT/Apache） | 开源（Apache 2.0） | 商业（免费层） |
| **规模** | 适合 **数千万‑上亿**（GPU） | 适合 **几万‑几百万**（单节点） | 适合 **数十亿**（多节点） |
| **索引类型** | 多样（Flat, IVF, HNSW, PQ, OPQ…） | 主要 HNSW + 简单 IVF | HNSW, IVF‑FLAT, IVF‑PQ, SPLADE 等 |
| **GPU 加速** | ✅（CUDA） | ❌（CPU） | ❌（完全托管，内部可能使用 GPU） |
| **元数据过滤** | 需要自行实现（可与 DB 组合） | ✅ 内置 | ✅ 内置 |
| **持久化** | 手动（`write_index`） | 自动（SQLite/DuckDB） | 自动（云存储） |
| **安全/多租户** | 自行搭建 | 本地安全，云版有限 | 完整 IAM、VPC、加密 |
| **成本** | 硬件成本（自己买机器） | 低（本地运行） | 按量付费（免费层） |
| **适合人群** | 研发、需要极致性能、或受合规约束 | 快速原型、RAG、团队协作 | 生产级大规模、需要 SLA 与弹性 |

FAISS：最灵活、最快速、完全本地，可自行掌控所有细节；适合技术团队、对性能有苛刻要求、或受数据合规限制的场景。
Chroma：轻量、开箱即用、与 LLM 框架高度集成，特别适合 RAG 原型开发和中小规模向量检索。
Pinecone：托管云服务，提供弹性伸缩、强大的安全与运维能力，是 大规模、生产环境 的首选。



In [71]:
# ------------------------------------------------------------
# 1️⃣ 导入 PromptTemplate（用于把模板字符串渲染成 Prompt 对象）
# ------------------------------------------------------------
from langchain_core.prompts import PromptTemplate

# ------------------------------------------------------------
# 2️⃣ 创建检索器（Retriever）
#    - vector_store：已经构建好的向量数据库（如 FAISS、Chroma、Pinecone 等）
#    - as_retriever() 会把向量库包装成 LangChain 标准的 Retriever 接口
# ------------------------------------------------------------
retriever = vector_store.as_retriever()

# ------------------------------------------------------------
# 3️⃣ 配置检索器的搜索参数
#    - search_kwargs 是一个 dict，里面的键值会直接传给底层的
#      向量库的相似度搜索函数
#    - k 表示 **返回最相似的前 k 条文档**（这里取 3 条）
# ------------------------------------------------------------
retriever.search_kwargs = {"k": 3}

# ------------------------------------------------------------
# 4️⃣ 执行检索
#    - invoke() 接收一个查询字符串，返回检索到的文档列表
#    - 每条文档通常是 LangChain 的 Document 对象，包含
#      .page_content（正文）和 .metadata（元信息）等属性
# ------------------------------------------------------------
docs = retriever.invoke("民法典进企业是什么？")

# ------------------------------------------------------------
# 5️⃣ 编写 LLM 提示模板（PromptTemplate）
#    - 这里使用多行字符串（"""）定义了一个“指令式”提示
#    - {info} 与 {question} 是占位符，稍后会被真实内容替换
# ------------------------------------------------------------
prompt_template = """
你是一个问答机器人。
你的任务是根据下述给定的已知信息回答用户问题。确保你的回复完全依据下述已知信息。不要编造答案。
如果下述已知信息不足以回答用户的问题，请直接回复”我无法回答您的问题”。
已知信息：{info}
用户问：
{question}
"""

# ------------------------------------------------------------
# 6️⃣ 把模板字符串转成 PromptTemplate 对象
#    - from_template() 会自动把占位符解析为模板变量
# ------------------------------------------------------------
template = PromptTemplate.from_template(prompt_template)

# ------------------------------------------------------------
# 7️⃣ 用检索到的文档和用户提问渲染 Prompt
#    - info 参数填入检索到的文档列表（LangChain 会把列表
#      自动转换为字符串，常见做法是将每篇文档的 .page_content
#      用换行或分隔符拼接起来）
#    - question 参数填入原始用户提问
# ------------------------------------------------------------
prompt = template.format(info=docs, question="民法典进企业是什么？")

# ------------------------------------------------------------
# 8️⃣ 调用大语言模型（LLM）生成答案
#    - llm 必须是已经实例化的 LLM 对象（OpenAI、Claude、ChatGLM…
#      任意支持 .invoke() 接口的模型）
#    - .invoke() 接收渲染好的 Prompt，返回一个包含
#      .content（生成文本）的响应对象
# ------------------------------------------------------------
response = llm.invoke(prompt)

# ------------------------------------------------------------
# 9️⃣ 打印模型的回答
# ------------------------------------------------------------
print(response.content)


根据提供的信息，2025年的“民法典宣传月”工作方案以“民法典进企业”为重点，旨在组织开展民法典系列普法宣传，全面提高企业诚信守法经营意识和能力，助力企业防范化解风险、依法维护自身合法权益，为促进经济高质量发展营造良好法治环境。

具体来说，“民法典进企业”活动要：
* 坚持以习近平新时代中国特色社会主义思想为指导，深入学习宣传民法典。
* 重点学习宣传平等、自愿、公平、诚信等基本原则，以及相关规定。
* 重点宣传优化营商环境、知识产权保护和科技创新、构建和谐劳动关系、中国企业“走出去”等与企业生产经营密切相关的法律法规。

因此，民法典进企业就是以宣传和贯彻实施民法典为核心，将法律宣传工作延伸到企业内部，帮助企业更好地了解和运用民法典，从而促进企业的健康发展。



# 6 使用Agent


In [75]:
# ===== 第一部分：定义 Retriever 工具 =====

from langchain.tools import tool

# @tool 装饰器：将函数转换为 LangChain 工具
# response_format="content_and_artifact"：设置返回格式
#   - "content"：发送给 LLM 的文本表示
#   - "artifact"：原始文档对象（保留元数据），供应用使用
@tool(response_format="content_and_artifact")
def retrieve_context(query: str):
    """获取信息，回答问题"""

    # 从向量数据库中相似性搜索
    # 使用给定的 query 查找最相似的 k=2 个文档
    retrieved_docs = vector_store.similarity_search(query, k=2)

    # 将检索到的文档格式化为字符串供 LLM 使用
    # 每个文档包含源信息和内容
    serialized = "\n\n".join(
        (f"Source: {doc.metadata}\nContent: {doc.page_content}")
        for doc in retrieved_docs
    )

    # 返回元组：(格式化文本, 原始文档)
    # - 格式化文本：LLM 会看到这个内容
    # - 原始文档：应用代码可以访问元数据
    return serialized, retrieved_docs


# ===== 第二部分：创建 LLM 模型 =====

from langchain.agents import create_agent
from langchain_openai import ChatOpenAI

# 初始化本地 Gemma 模型（通过 LM Studio）
# 注意：这里配置的是本地模型，不是 OpenAI 官方 API
llm = ChatOpenAI(
    model="google/gemma-3n-e4b",        # 使用的模型 ID
    openai_api_key="lm-studio",          # API 密钥（本地可设为任意值）
    openai_api_base="http://localhost:1234/v1",  # 本地服务地址
    temperature=0                         # 温度为 0：确定性输出，不使用随机性
)


# ===== 第三部分：创建 Agent =====

# create_agent：LangChain v1 标准 agent 创建函数
# 基于 ReAct 循环（推理 + 行动）
agent = create_agent(
    model=llm,                               # 使用上面定义的 LLM
    tools=[retrieve_context],                # 工具列表：包含我们的 retriever 工具
    system_prompt="你是一个有帮助的民法典助手。", # 系统提示词
)

# Agent 执行流程：
# 1. 接收用户输入
# 2. 调用 llm 进行推理
# 3. LLM 决定是否调用 retrieve_context 工具
# 4. 如果调用，获取检索结果
# 5. 基于检索结果生成最终答案
# 6. 重复 2-5，直到 LLM 认为可以提供最终答案


# ===== 第四部分：执行 Agent =====

# 调用 agent，传入用户问题
result = agent.invoke({
    "messages": [
        {"role": "user", "content": "民法典进企业工作要求是什么？"}
    ]
})
print( result["messages"][-1].content)

# result 是一个字典，包含 agent 的输出
# 可以通过 result["messages"][-1].content 获取最终答案

民法典进企业工作要求主要包括：

1. **加强组织领导和协同配合：** 党委宣传部、司法行政部门要发挥牵头作用，与行业主管部门协调配合，落实普法责任。
2. **注重宣传内容和形式：** 围绕企业实际需求，精心组织有特色、接地气的宣传活动。结合传统方式和现代化手段，运用各种媒体（包括新兴媒体如人工智能、大数据、短视频）进行宣传。
3. **强调服务企业：** 坚持在服务企业中开展法治宣传，不干扰企业正常生产经营秩序，不增加企业负担。
4. **重点宣传内容：** 学习宣传习近平法治思想和民法典，重点强调平等、自愿、公平、诚信等基本原则。

总而言之，民法典进企业工作旨在提高企业依法经营的意识和能力，助力企业防范化解风险，维护自身合法权益。


